In [1]:
import pandas as pd
from collections import OrderedDict
from collections import Counter
import sys
import os
from glob import glob
import sqlite3

In [2]:
paths_1973 = glob("../../../../../Documents/data/1973-09-to-2014-06/non-dod/status/*.txt")
paths_2014 = glob("../../../../../Documents/data/2014-09-to-2016-09/non-dod/status/*.txt")
paths_2016 = glob("../../../../../Documents/data/2016-12-to-2017-03/non-dod/status/*.txt")

In [3]:
fwf_columns = OrderedDict([
    ('Pseudo_ID', (0, 9)),
    ('Name', (9, 32)),
    ('File_Date', (32, 40)),
    ('SubAgency', (40, 44)),
    ('Duty_Station', (44, 53)),
    ('Age_Range', (53, 59)),
    ('Education_Level', (59, 61)),
    ('Pay_Plan', (61, 63)),
    ('Grade', (63, 65)),
    ('LOS_Level', (65, 71)),
    ('Occupation', (71, 75)),
    ('PATCO', (75, 76)),
    ('Adjusted_Basic_Pay', (76, 82)),
    ('Supervisory_Status', (82, 83)),
    ('TOA', (83, 85)),
    ('Work_Schedule', (85, 86)),
    ('NSFTP_Indicator', (86, 87))
])

In [4]:
cols_2014 = [
    "Last_Name",
    "First_Name",
    "File_Date",
    "Agency",
    "SubAgency",
    "State",
    "Age_Range",
    "YSD_Range",
    "Education_Level",
    "Pay_Plan",
    "Grade",
    "LOS_Level",
    "Occupation",
    "PATCO",
    "Adjusted_Basic_Pay",
    "Supervisory_Status",
    "TOA",
    "Work_Schedule",
    "NSFTP_Indicator",
]

In [5]:
cols_2014 = [
    "Last_Name",
    "First_Name",
    "File_Date",
    "Agency",
    "SubAgency",
    "State",
    "Age_Range",
    "YSD_Range",
    "Education_Level",
    "Pay_Plan",
    "Grade",
    "LOS_Level",
    "Occupation",
    "PATCO",
    "Adjusted_Basic_Pay",
    "Supervisory_Status",
    "TOA",
    "Work_Schedule",
    "NSFTP_Indicator",
]

In [6]:
conn = sqlite3.connect("salaries.db")
cur = conn.cursor()

In [7]:
def process_fwf(path):
    slices = fwf_columns.values()
    occupations = set(["1863", "0512", "1169", "1825", "2152", "2154"])
    with open(path) as f:
        rows = []
        for line in f:
            row = [line[start:end].strip() for start, end in slices]
            # Check if the occupation of the person follows what we want, and has a valid adjusted basic pay
            if row[10] in occupations and row[12] not in ["******", ""]:
                rows.append(row)
        return pd.DataFrame(rows, columns=list(fwf_columns.keys()))\
    .drop_duplicates(subset=["Pseudo_ID"])\
     .replace({
         "Age_Range": {"UNSP": "NA", " ": "NA", "": "NA"},
         "Education_Level": {"**": "NA", " ": "NA", "": "NA"},
         "Pay_Plan": {"**": "NA", " ": "NA", "": "NA"},
         "Grade": {"**": "NA", " ": "NA", "": "NA"},
         "Supervisory_Status": {"*": "NA", " ": "NA", "": "NA"},
         "TOA": {"**": "NA", " ": "NA", "": "NA"},
         "Work_Schedule": {"*": "NA", " ": "NA", "": "NA"}
     })\
    .assign(**{
        "File_Date": lambda x: x["File_Date"].apply(lambda y: int(y[:-2])),
         "Adjusted_Basic_Pay": lambda x: x["Adjusted_Basic_Pay"].apply(lambda y: int(float(y))),
         "NSFTP_Indicator": lambda x: x["NSFTP_Indicator"].apply(lambda y: pd.to_numeric(y, errors="coerce"))
    })\
    .drop(["Pseudo_ID", "Name", "Duty_Station"], axis=1)\
    .replace({
        "Age_Range": {"65-69": "65+", "70-74": "65+", "75+": "65+"},
        "LOS_Level": {"UNSP": "NA"}
    })

In [8]:
for path in paths_1973:
    table_1973 = process_fwf(path)
    table_1973.to_sql(name="data", con=conn, if_exists="append", index=False)

In [9]:
table_2014 = pd.concat(
    [pd.read_csv(
        path,
        sep="|",
        low_memory=False,
        names=cols_2014
    )\
     .assign(**{
         "SubAgency": lambda x: x["SubAgency"].apply(lambda y: y.split("-")[0]),
         "LOS_Level": lambda x: x["LOS_Level"].apply(lambda y: y.replace(" years", "").replace(" - ", "-")),
         "Occupation": lambda x: x["Occupation"].apply(lambda y: y.split("-")[0])
     })\
     .replace({
         "Education_Level": {"": "NA", "*": "NA", " ": "NA"}
     })\
     .query("Occupation in ['1863', '0512', '1169', '1825', '2152', '2154'] and Adjusted_Basic_Pay != '.'")\
     .assign(**{
         "File_Date": lambda x: x["File_Date"].apply(lambda y: pd.to_numeric(y)),
         "Adjusted_Basic_Pay": lambda x: x["Adjusted_Basic_Pay"].apply(lambda y: int(float(y))),
         "NSFTP_Indicator": lambda x: x["NSFTP_Indicator"].apply(lambda y: pd.to_numeric(y, errors="coerce"))
     })\
     .drop(["Last_Name", "First_Name", "Agency", "State", "YSD_Range"], axis=1)\
     .replace({
         "Age_Range": {"Less than 20": "15-19", "65 or more": "65+"},
         "LOS_Level": {"35 or more": "35+", "Less than 1 year": "< 1"}
     }) 
     for path in paths_2014])\
.reset_index(drop=True)

In [10]:
table_2014.to_sql(name="data", con=conn, if_exists="append", index=False)

In [11]:
table_2016 = pd.concat(
    [pd.read_csv(path, 
                 sep=";", 
                 low_memory=False,
                 names=cols_2014
                )\
     .assign(**{
         "SubAgency": lambda x: x["SubAgency"].apply(lambda y: y.split("-")[0]),
         "YSD_Range": lambda x: x["YSD_Range"].apply(lambda y: y.replace(" years", "").replace(" - ", "-")),
         "Education_Level": lambda x: x["Education_Level"].apply(lambda y: y.split("-")[0]),
         "Pay_Plan": lambda x: x["Pay_Plan"].apply(lambda y: y.split("-")[0]),
         "LOS_Level": lambda x: x["LOS_Level"].apply(lambda y: y.replace(" years", "").replace(" - ", "-")),
         "Occupation": lambda x: x["Occupation"].apply(lambda y: y.split("-")[0]),
         "Supervisory_Status": lambda x: x["Supervisory_Status"].apply(lambda y: y.split("-")[0]),
         "TOA": lambda x: x["TOA"].apply(lambda y: y.split("-")[0]),
         "Work_Schedule": lambda x: x["Work_Schedule"].apply(lambda y: y.split("-")[0]),
         "NSFTP_Indicator": lambda x: x["NSFTP_Indicator"].apply(lambda y: y.split("-")[0])
     })\
     .replace({
         "Education_Level": {"": "NA", "*": "NA"}
     })\
     .query("Occupation in ['1863', '0512', '1169', '1825', '2152', '2154'] and Adjusted_Basic_Pay != '.'")\
     .assign(**{
         "File_Date": lambda x: x["File_Date"].apply(lambda y: int(y)),
         "Adjusted_Basic_Pay": lambda x: x["Adjusted_Basic_Pay"].apply(lambda y: int(float(y))), 
         "NSFTP_Indicator": lambda x: x["NSFTP_Indicator"].apply(lambda y: pd.to_numeric(y, errors="coerce"))
     })\
     .drop(["Last_Name", "First_Name", "Agency", "State", "YSD_Range"], axis=1)\
     .replace({
         "Age_Range": {"Less than 20": "15-19", "65 or more": "65+"},
         "LOS_Level": {"35 or more": "35+", "Less than 1 year": "< 1"}
     }) 
     for path in paths_2016])\
.reset_index(drop=True)

In [12]:
table_2016.to_sql(name="data", con=conn, if_exists="append", index=False)

In [13]:
conn.commit()
conn.close()